_**```author 'Kashirin Alex <kashirin.alex@gmail.com>'```**_

# Multi Band Audio Extractor

## Main-Functionalities

* Read Audio Fle
* Extract desired Bands from an Audio Data
* Preview plot the Audio Data  (wave-amplitude)
* Write Audio Data to file 
* Recognize Dense Peaks of the Audio Data of specified samples duration


## Conclusion - Result
The 60 seconds of noise/polluted audio-data summarised in couple(`r.recognize_dense_peaks(data, n_samples=48000 * 3)` the n_samples ) of seconds of pure-sound.








In [ ]:
import IPython.display as ipd # for playing audio

## The MultiBandAudioExtractor class

### Fields/Attributes
* filepath -- the audio file-path
* data -- the original audio-data
* samplerate -- the audio-data sample-rate
* milliseconds -- the audio-data duration
* total_samples -- the audio-data total-samples


### Methods
* extract_ifft -- returns the extracted audio-data in ndarray
* write1 -- write audio data to file with SoundFile
* write2 -- write audio data with `wave` module `.wav` in int24 bits-width
* preview -- plot (show/write) audio-data, x-samples y-amplitude 

### Static Methods
* preamp -- Preamplify the signal (it is as well used within the class)
* recognize_dense_peaks -- returns data-index in tuple(start, finish)
* get_ranges_name -- helper to create file/name from ranges ```[(start, finish), ]```
* 


In [ ]:
# -- coding: utf-8 --
__author__ = 'Kashirin Alex <kashirin.alex@gmail.com>'


import soundfile as sf
import matplotlib.pyplot as plt
import numpy as np
from scipy import fftpack
import math
#
import wave
import struct
#
import os
import warnings
warnings.simplefilter('ignore', np.ComplexWarning)
#


class MultiBandAudioExtractor(object):

    POW_23 = math.pow(2, 23)

    def __init__(self, _filepath):
        self.filepath = _filepath

        with open(_filepath, 'rb') as f:
            self.data, self.samplerate = sf.read(f)
        self.total_samples = len(self.data)
        self.preamp(self.data)

        self.milliseconds = int(float(self.total_samples/self.samplerate)*1000)
        #

    @staticmethod
    def preamp(_data):
        _preamp = 0.99 / (1 - min([1.0 - max(_data), 1.0 + min(_data)]))
        if _preamp > 1.0:
            for n in range(0, len(_data)):
                _data[n] += 1.0
                _data[n] *= _preamp
                _data[n] -= _preamp
            mid = sum(_data) / len(_data)
            for n in range(0, len(_data)):
                _data[n] -= mid
            _post_preamp = 0.99 / (1 - min([1.0 - max(_data), 1.0 + min(_data)]))
            print('preamp:', _preamp, 'mid:', mid, 'remain-preamp:', _post_preamp)
        #

    def extract_ifft(self, _freq_bands=None):
        fft = fftpack.fft(self.data)
        if _freq_bands is None:
            _data = fftpack.ifft(fft)
            self.preamp(_data)
            return _data

        sample_freq = fftpack.fftfreq(len(fft), d=1.0 / self.samplerate)
        _data = None
        for start, finish in _freq_bands:
            fft_tmp = fft.copy()
            if start:
                fft_tmp[np.abs(sample_freq) < start] = 0.0
            if finish:
                fft_tmp[np.abs(sample_freq) > finish] = 0.0
            if _data is None:
                _data = fftpack.ifft(fft_tmp)
            else:
                tmp = fftpack.ifft(fft_tmp)
                for n in range(0, len(_data)):
                    _data[n] += tmp[0]
        if _data is None:
            return []
        self.preamp(_data)
        return _data
        #

    @staticmethod
    def get_ranges_name(_ranges):
        return str(_ranges if _ranges is None else '_'.join([str(_s)+'-'+str(_f) for _s, _f in _ranges]))
        #

    def write1(self, _filename, _data, _samples_range=None):
        _s, _f = (0, len(_data) - 1) if _samples_range is None else _samples_range
        with sf.SoundFile(_filename, 'w+', channels=1, samplerate=self.samplerate) as fd:
            fd.write([float(_data[n]) for n in range(_s, _f+1)])
        #

    def write2(self, _filename, _data, _samples_range=None):
        _s, _f = (0, len(_data) - 1) if _samples_range is None else _samples_range
        wf = wave.open(_filename, mode='w')
        wf.setnchannels(1)
        wf.setsampwidth(3)
        wf.setframerate(self.samplerate)
        buffer = bytearray()
        for n in range(_s, _f + 1):
            buffer += bytearray(struct.pack('<i', int(float(_data[n]) * self.POW_23))[:-1])
        wf.writeframes(bytes(buffer))
        wf.close()
        #

    def preview(self, _filename, _data, _samples_range=None):
        if _samples_range is None:
            _s = 0
            _f = len(_data)
            _width = int(_f/(self.samplerate/2))
            _f -= 1
        else:
            _width = 60
            _s, _f = _samples_range
            _f1 = len(_data)
            if _f >= _f1:
                _f = _f1 - 1

        plt.rcParams['agg.path.chunksize'] = (_f-_s) * 2
        plt.figure(figsize=(_width, 10))
        x = np.linspace(0.00, _f-_s, num=_f-_s, endpoint=False)

        plt.plot(x, _data[_s:_f])
        plt.xlabel('Time (samples)')
        plt.ylabel('Amplitude ($Unit$)')
        if _filename:
            plt.savefig(_filename)
        else:
            plt.show()
        plt.close()
        #

    @staticmethod
    def recognize_dense_peaks(_data, n_samples=800):
        offset = int(n_samples/2)
        last_idx = len(_data) - 1

        tmp = np.absolute(_data)
        _mean = tmp.mean()
        _max = tmp.max()
        idx = None
        density = 0
        for n in range(0, len(tmp)):
            if _mean < tmp[n] < _max:
                _s = (n - offset) if n > offset else 0
                _f = last_idx if last_idx <= n + offset else (n + offset)
                _sum = tmp[_s:_f].sum()
                if density < _sum:
                    density = _sum
                    idx = n
        return None if idx is None \
            else ((idx - offset) if idx > offset else 0,
                  last_idx if last_idx <= (idx + offset) else (idx + offset))
        #

#



## Process Extraction

### Define the Actions for Proceeding of the Name and the Frequency-Bands

In [ ]:

def process_extraction(r, name, freq_bands):
    filepath = r.filepath.split('/')[-1].split('.')[0] + '-'
    filepath += name + '_bands(' + r.get_ranges_name(freq_bands) + ')'
    
    print('Extracting ', filepath)
    
    ## Extract the corresponding frequency-bands
    data = r.extract_ifft(freq_bands)
    # Write
    r.write2(filepath + '.wav', data)
    
    # Play
    ipd.Audio(filepath + '.wav')
   
    # Plot
    r.preview(None, data, [0, 2048])
        
    
    ## Recognize Dense Peaks
    dense_range = r.recognize_dense_peaks(data, n_samples=48000 * 3)
    
    filepath += '_dense-samples(' + r.get_ranges_name([dense_range]) + ')'
    
    print('making ', filepath, "dense_range:", dense_range, "second-start:", dense_range[0]/r.samplerate)
    
    if dense_range:
        # Write
        r.write2(filepath + '.wav', data, dense_range)
        
        # Play
        ipd.Audio(filepath + '.wav')

        # Plot
        r.preview(None, data, dense_range)
    




## Process File

#### Initialize the ```MultiBandAudioExtractor``` with the filepath
#### Data summary
#### Define & Proceed to Extract:
* ##### Birds
   

In [ ]:


def process_file(filepath):
    print("Processing", filepath)
    
    r = MultiBandAudioExtractor(filepath)

    print(r.filepath,
          'sample-rate:', r.samplerate,
          'milliseconds:', r.milliseconds,
          'total-samples:', r.total_samples)
    
    # Extract Birds
    process_extraction(r, 'Birds', [
        (2900, 3100),
        (4700, 5000),
        (7100, 7300),
        (10800, 11100)
    ])
    
    
    # for name, freq_bands in [
        # ('200HzTest', [(200, 201)]),
        #('Frogs', [
        #    (1200, 1300),
        #    (1700, 1800),
        #    (2900, 2920),
        #])
    # ]:
        
#

## Read the Directories for Audio Samples

In [ ]:

numfiles = 0
files = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    numfiles += len(filenames)
    for filename in filenames:
        if filename.endswith(".flac"):
            files.append(os.path.join(dirname, filename))

print("Total Audio Samples", numfiles)
for filepath in files[0:10]:
    process_file(filepath)